In [9]:
ps = [('Pepe','M',32),('Veronica','F',37),('Juan','M', 28)]
print ps

[('Pepe', 'M', 32), ('Veronica', 'F', 37), ('Juan', 'M', 28)]


In [3]:
rdd = sc.parallelize(ps)

In [5]:
rdd.collect()

[('Pepe', 'M', 32), ('Veronica', 'F', 37), ('Juan', 'M', 28)]

In [19]:
sum([x[2] for x in ps])/3.0

32.333333333333336

In [13]:
import math

In [17]:
rdd.map(lambda x: x[2]).collect()

[32, 37, 28]

In [18]:
rdd.map(lambda x: x[2]).mean()

32.333333333333336

In [21]:
kv = rdd.map(lambda x: (x[1],x[2]))

In [22]:
kv.collect()

[('M', 32), ('F', 37), ('M', 28)]

In [24]:
kv.groupByKey().collect()

[('M', <pyspark.resultiterable.ResultIterable at 0x7f0ea16b6e90>),
 ('F', <pyspark.resultiterable.ResultIterable at 0x7f0ea16b6350>)]

In [33]:
kv.groupByKey().map(lambda x: (x[0],sum(x[1]))).collect()

[('M', 60), ('F', 37)]

In [35]:
kv.groupByKey().mapValues(lambda vs: sum(vs)).collect()

[('M', 60), ('F', 37)]

In [36]:
# mapValues, considera que recibe una tupla de clave y lista de valores, y considera únicamente los valores como la lista 
# de valores
kv.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).collect()

[('M', 30), ('F', 37)]

In [41]:
# Trabajamos con datos
!ls

data	      PySpark-01_clase.ipynb	 shakespeare.txt
data.tar.bz2  PySpark-01.ipynb
originales    Segunda clase Spark.ipynb


In [44]:
ls = sc.textFile('data/coupon150720.csv') # Se puede añadir el número de particiones, si añades un 2º parámetro, 100

In [45]:
# Recuperación de 5 líneas, no se garantiza que sean las 5 primeras del archivo (sobre todo en distribuido)
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [46]:
# Recuperacion de la primera línea
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [48]:
ls.count()

1232662

In [ ]:
# con - ls.cache() el siguiente comando lo guarda 

In [ ]:
# controla cómo hace la operación cache, a dónde se guarda: Memoria, Disco, ... Por defecto a Memoria
ls.persist()

In [49]:
ls.getNumPartitions()

3

In [50]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [59]:
# En vuelos Iberia, cuantos pasajeros ha tenido
# Nos quedamos con la operadora del vuelo 5º campo
ls_split = ls.map(lambda x: x.split(",")[5])

In [61]:
# Nos quedamos co los vuelos de Iberia
ls_split.filter(lambda x: x=="IB").count()

26158

In [76]:
# Dinero realizado por los vuelos de Iberia
ls_split = ls.map(lambda x: x.split(","))
ls_split_tuplas = ls_split.map(lambda x: (x[5],float(x[6])))
ls_split_tuplas_filter = ls_split_tuplas.filter(lambda x: x[0]=="IB")
ls_split_tuplas_filter.groupByKey().mapValues(lambda x: sum(x)).collect()

[(u'IB', 2828044.419999916)]

In [82]:
def AerolineaDinero(l):
    elements = l.split(",")
    aerolinea = elements[5]
    dinero = float(elements[6])
        
    return (aerolinea, dinero)

dineros = ls.map(AerolineaDinero).filter(lambda x: x[0]=="IB")
#dineros.take(3)
#dineros.groupByKey().mapValues(lambda x: sum(x)).collect()
#dineros.stats()
dineros.map(lambda x:x[1]).sum()


[(u'IB', 2828044.419999916)]

In [94]:
def getRutas(l):
    elements = l.split(",")
    origen = elements[2]
    destino = elements[3]
    dinero = float(elements[6])
    ruta = origen + " " + destino
        
    return (ruta, dinero)

rutas = ls.map(getRutas)


In [96]:
rutas.sortBy(lambda x: x[1], False).take(5)

(u'LGA EWR', 0.0)

In [ ]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5)

In [98]:
rutas.cache()
rutas.filter(lambda x: x[1] < 7500).groupByKey().take(3)

[(u'BKK CSX', <pyspark.resultiterable.ResultIterable at 0x7f0ea16771d0>),
 (u'BOJ RIX', <pyspark.resultiterable.ResultIterable at 0x7f0ea16c7a90>),
 (u'CDG MSQ', <pyspark.resultiterable.ResultIterable at 0x7f0ea15ff650>)]

In [100]:
rutas.filter(lambda x: x[1] < 7500).groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'BKK CSX', 152.81), (u'BOJ RIX', 328.62), (u'CDG MSQ', 141.49)]

In [102]:
rutas.filter(lambda x: x[1] < 7500).groupByKey().mapValues(lambda vs: max(vs)).sortBy(lambda x: x[1], False).take(5)

[(u'BLA MIA', 7490.33),
 (u'CCS MIA', 7415.0),
 (u'MIA CCS', 7415.0),
 (u'RUH LAX', 7401.94),
 (u'DXB MAA', 7343.27)]

In [103]:
rutas.filter(lambda x: x[1] < 7500).groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).sortBy(lambda x: x[1], False).take(5)

[(u'MEC BLA', 5164.355),
 (u'BLA MEC', 5164.355),
 (u'CCS MIA', 4350.516296296297),
 (u'MIA CCS', 4186.95),
 (u'PPT NRT', 3751.5499999999997)]

In [105]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [108]:
def getDatoRuta (l):
    linea = l.split(",");
    ruta = linea[2] + " " + linea[3]
    al = linea[5]
    dinero = float(linea[6])
    clase = linea[9]
    
    return ((ruta, al, clase), dinero)

In [109]:
getDatoRuta(ls.first())

((u'MAA AUH', u'9W', u'H'), 56.79)

In [123]:
# Definicion de valor límite medio
# rutas.filter(lambda x: x[1] < 7500)
import math
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    # vs.map(lambda x: math.log(x)) - Es lo mismo que la primera linea
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    varianza = sum(desv)/len(desv)
    desvStd = math.sqrt(varianza)
    
    return math.exp(media + 2*desvStd)-1

In [113]:
valorLimite([12,3,4,1,2,4])

14.668153608222282

In [116]:
rutas = ls.map(getDatoRuta)
rutas.take(5)

[((u'MAA AUH', u'9W', u'H'), 56.79),
 ((u'AUH CDG', u'9W', u'H'), 84.34),
 ((u'CJB MAA', u'9W', u'H'), 60.0),
 ((u'DEL DXB', u'9W', u'S'), 160.63),
 ((u'AUH IXE', u'9W', u'V'), 152.46)]

In [124]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    
    if len(f) >= 10:
        return f
    else:
        return []

rutas.groupByKey().mapValues(filtraCuponesExtremos).take(2)

[((u'OPO LGW', u'TP', u'L'), []), ((u'ICN HRB', u'OZ', u'W'), [])]

In [128]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)
grupos.mapValues(lambda vs: sum(vs)/max(len(vs),1)).take(2)

[((u'OPO LGW', u'TP', u'L'), 0), ((u'ICN HRB', u'OZ', u'W'), 0)]

In [129]:
grupos.mapValues(lambda vs: sum(vs)/max(len(vs),1)).sortBy(lambda x:x[1],False).take(5)

[((u'MIA CCS', u'S3', u'S'), 7826.25),
 ((u'CCS MIA', u'S3', u'B'), 5550.0),
 ((u'CCS MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH FRA', u'LH', u'F'), 4792.983),
 ((u'LHR DOH', u'QR', u'F'), 4275.98923076923)]

In [127]:
# Eliminar grupos pequeños - Se puede también hacer a través de la funcion filtraCuponesExtremo
grupos_filtrados = grupos.filter(lambda x: len(x[1]) >= 10)
grupos_filtrados.mapValues(lambda vs: sum(vs)/max(len(vs),1)).sortBy(lambda x:x[1],False).take(5)

[((u'MIA CCS', u'S3', u'S'), 7826.25),
 ((u'CCS MIA', u'S3', u'B'), 5550.0),
 ((u'CCS MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH FRA', u'LH', u'F'), 4792.983),
 ((u'LHR DOH', u'QR', u'F'), 4275.98923076923)]

In [131]:
# Cuantos valores diferentes hay en los datos
ls.map(lambda x : x.split(",")[4]).distinct().count()

359

In [132]:
ls.map(lambda x : x.split(",")[7]).distinct().collect()

[u'USD']

In [134]:
ls.map(lambda x : x.split(",")[4]).distinct().saveAsTextFile("data/fichero_salida.txt")